In [ ]:
# python3 -m venv .venv
# source .venv/bin/activate
# pip install mellow_strategy_sdk

In [ ]:
from mellow_sdk.primitives import Pool, POOLS
from mellow_sdk.data import RawDataUniV3
from mellow_sdk.strategies import AbstractStrategy
from mellow_sdk.backtest import Backtest
from mellow_sdk.viewers import RebalanceViewer, UniswapViewer, PortfolioViewer
from mellow_sdk.positions import BiCurrencyPosition, UniV3Position

### Choose a pool

In [ ]:
pool_num = 1
pool = Pool(
    tokenA=POOLS[pool_num]['token0'],
    tokenB=POOLS[pool_num]['token1'],
    fee=POOLS[pool_num]['fee']
)

### Get data

In [ ]:
data = RawDataUniV3(pool, 'data', reload_data=False).load_from_folder()

### Create strategy

In [ ]:
class UniV3Passive(AbstractStrategy):
    """
    ``UniV3Passive`` is the passive strategy on UniswapV3, i.e. mint one interval and wait.

    Attributes:
        lower_price: Lower bound of the interval
        upper_price: Upper bound of the interval
        gas_cost: Gas costs, expressed in currency
        pool: UniswapV3 Pool instance
        name: Unique name for the instance
    """
    def __init__(self,
                 lower_price: float,
                 upper_price: float,
                 pool: Pool,
                 gas_cost: float,
                 name: str = None,
                 ):
        super().__init__(name)
        self.lower_price = lower_price
        self.upper_price = upper_price

        self.fee_percent = pool.fee.percent
        self.gas_cost = gas_cost
        self.swap_fee = pool.fee.percent

    def rebalance(self, *args, **kwargs) -> str:
        # record is row of historic data
        record = kwargs['record']
        
        # portfolio managed by the strategy
        portfolio = kwargs['portfolio']
        price_before, price = record['price_before'], record['price']

        is_rebalanced = None

        if len(portfolio.positions) == 0:
            self.create_uni_position(portfolio=portfolio, price=price)
            is_rebalanced = 'mint'

        if 'UniV3Passive' in portfolio.positions:
            uni_pos = portfolio.get_position('UniV3Passive')
            
            # collect fees from uni
            uni_pos.charge_fees(price_before, price)

        return is_rebalanced

    def create_uni_position(self, portfolio, price):
        x = 1 / price
        y = 1
        
        # create biccurency positions for swap
        bi_cur = BiCurrencyPosition(
            name=f'main_vault',
            swap_fee=self.swap_fee,
            gas_cost=self.gas_cost,
            x=x,
            y=y,
            x_interest=None,
            y_interest=None
        )
        # create uni interval
        uni_pos = UniV3Position(
            name=f'UniV3Passive',
            lower_price=self.lower_price,
            upper_price=self.upper_price,
            fee_percent=self.fee_percent,
            gas_cost=self.gas_cost,
        )

        portfolio.append(bi_cur)
        portfolio.append(uni_pos)

        # get tokens amount to swap
        dx, dy = uni_pos.aligner.get_amounts_for_swap_to_optimal(
            x, y, swap_fee=bi_cur.swap_fee, price=price
        )
        
        # swap
        if dx > 0:
            bi_cur.swap_x_to_y(dx, price=price)
        if dy > 0:
            bi_cur.swap_y_to_x(dy, price=price)
        
        x_uni, y_uni = uni_pos.aligner.get_amounts_after_optimal_swap(
            x, y, swap_fee=bi_cur.swap_fee, price=price
        )
        
        # withdraw tokens from bicurrency
        bi_cur.withdraw(x_uni, y_uni)
        
        # deposit tokens to uni
        uni_pos.deposit(x_uni, y_uni, price=price)

### Backtest

In [ ]:
univ3_passive = UniV3Passive(
    lower_price=data.swaps['price'].min(),
    upper_price=data.swaps['price'].max(),
    pool=pool,
    gas_cost=0.,
    name='passive'
)

bt = Backtest(univ3_passive)
portfolio_history, rebalance_history, uni_history = bt.backtest(data.swaps)

### Visualise

In [ ]:
rv = RebalanceViewer(rebalance_history)
uv = UniswapViewer(uni_history)
pv = PortfolioViewer(portfolio_history, pool)

# Draw portfolio stats, like value, fees earned, apy
fig1, fig2, fig3, fig4, fig5, fig6 = pv.draw_portfolio()

# Draw Uniswap intervals
intervals_plot = uv.draw_intervals(data.swaps)

# Draw rebalances
rebalances_plot = rv.draw_rebalances(data.swaps)

# Calculate df with portfolio stats
stats = portfolio_history.calculate_stats()

In [ ]:
intervals_plot.show(render='svg')

In [ ]:
rebalances_plot.show(render='svg')

In [ ]:
fig2.show(render='svg')

In [ ]:
fig4.show(render='svg')

In [ ]:
fig6.show(render='svg')

In [ ]:
stats.tail(2)

In [ ]:
stats.tail(2).to_csv('getting_started_stats.csv')

In [ ]:
rebalances_plot.write_image('getting_started_rebalances.png')
intervals_plot.write_image('getting_started_intervals.png')
fig2.write_image('getting_started_fig2.png')
fig4.write_image('getting_started_fig4.png')
fig6.write_image('getting_started_fig6.png')